In [1]:
from ngsolve import *
from ngsolve.la import EigenValues_Preconditioner

mesh = Mesh(unit_square.GenerateMesh(maxh=0.3))

fes = H1(mesh,order=0, dirichlet=".*", autoupdate=True)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx)

importing NGSolve-6.2.2301


In [2]:
class MGPreconditioner(BaseMatrix):
    def __init__ (self, fes, level, mat, coarsepre):
        super().__init__() # Inheriting BaseMatrix class attributes
        self.fes = fes
        self.level = level
        self.mat = mat
        self.coarsepre = coarsepre
        
        # For level = 0 do direct inverse, else smooth via iterations of GS
        if level > 0:
            self.localpre = mat.CreateSmoother(fes.FreeDofs()) # Point-Jacobi
        else:
            self.localpre = mat.Inverse(fes.FreeDofs())

    def Mult (self, d, w):
        if self.level == 0:
            w.data = self.localpre * d # A^{-1} * d.
            return

        prol = self.fes.Prolongation().Operator(self.level) # Interpolation operator to specified level.

        w[:] = 0
        for i in range(1):
            self.localpre.Smooth(w,d) # One step of point gauss-seidel for solution vector w and basevector d
            res  = d - self.mat * w # Computing residual as d - Aw
            
        w += (prol @ self.coarsepre @ prol.T) * res # Coarse grid correction 
        self.localpre.SmoothBack(w,d) # going back up the heirarchy.


    def Shape (self):
        return self.localpre.shape
    def CreateVector (self, col):
        return self.localpre.CreateVector(col)
    
print('Done')

Done


In [3]:
a.Assemble()
pre = MGPreconditioner(fes, 0, a.mat, None)

for l in range(9):
    mesh.Refine()
    a.Assemble()
    pre = MGPreconditioner(fes,l+1, a.mat, pre)
    lam = EigenValues_Preconditioner(a.mat, pre)
    print("ndof=%7d:  minew=%.4f  maxew=%1.4f  Cond# = %5.3f"
          %(fes.ndof, lam[0], lam[-1], lam[-1]/lam[0]))

ndof=     61:  minew=0.7547  maxew=0.9983  Cond# = 1.323
ndof=    217:  minew=0.5814  maxew=0.9970  Cond# = 1.715
ndof=    817:  minew=0.5444  maxew=0.9968  Cond# = 1.831
ndof=   3169:  minew=0.5070  maxew=0.9970  Cond# = 1.966
ndof=  12481:  minew=0.4870  maxew=0.9964  Cond# = 2.046
ndof=  49537:  minew=0.4702  maxew=0.9957  Cond# = 2.118
ndof= 197377:  minew=0.4650  maxew=0.9949  Cond# = 2.140
ndof= 787969:  minew=0.4599  maxew=0.9944  Cond# = 2.162
ndof=3148801:  minew=0.4583  maxew=0.9937  Cond# = 2.168


In [4]:
f = LinearForm(1*v*dx).Assemble()
gfu = GridFunction(fes)
from ngsolve.krylovspace import CGSolver
inv = CGSolver(mat=a.mat, pre=pre, printrates=True)
gfu.vec.data = inv * f.vec

CG iteration 1, residual = 0.17485441072068886     
CG iteration 2, residual = 0.010217057003015048     
CG iteration 3, residual = 0.001256139314100235     
CG iteration 4, residual = 0.00017274545836876902     
CG iteration 5, residual = 2.7819740145590537e-05     
CG iteration 6, residual = 4.9176954776254025e-06     
CG iteration 7, residual = 9.23274395179222e-07     
CG iteration 8, residual = 1.8379122279574705e-07     
CG iteration 9, residual = 3.4689867880548826e-08     
CG iteration 10, residual = 6.500901537377572e-09     
CG iteration 11, residual = 1.1574897514971914e-09     
CG iteration 12, residual = 2.600590330238873e-10     
CG iteration 13, residual = 5.797339811742631e-11     
CG iteration 14, residual = 9.181963350169727e-12     
CG iteration 15, residual = 1.7777025594254866e-12     
CG iteration 16, residual = 3.7355694702182974e-13     
CG iteration 17, residual = 6.856536116363762e-14     
